In [2]:
# Simple CNN model for CIFA-10
import numpy as np
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

In [3]:
# fix randow seed for reporducibility
seed =7
np.random.seed(seed)

In [4]:
#load data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [5]:
# normalize inputs from 0-255 to 0.0-1.0
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

In [7]:
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [8]:
# 1. Convolutional input layer, 32 feature maps with a size of 3×3, a rectifier activation function and a weight constraint of max norm set to 3.
# 2. Dropout set to 20%.
# 3. Convolutional layer, 32 feature maps with a size of 3×3, a rectifier activation function and a weight constraint of max norm set to 3.
# 4. Max Pool layer with size 2×2.
# 5. Flatten layer.
# 6. Fully connected layer with 512 units and a rectifier activation function.
# 7. Dropout set to 50%.
# 8. Fully connected output layer with 10 units and a softmax activation function.

#create the model
model = Sequential()
model.add(Conv2D(32, (3,3), input_shape=(3, 32, 32), padding='same', activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Conv2D(32,(3,3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
# compile model
epochs = 25
lrate = 0.01
decay = lrate/ epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_3 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 16, 16)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               4194816   
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
__________

In [ ]:
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=32)
# Finbal evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1] * 100))

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 449s - loss: 1.7174 - acc: 0.3772 - val_loss: 1.3951 - val_acc: 0.4929
Epoch 2/25
50000/50000 [==============================] - 434s - loss: 1.3374 - acc: 0.5196 - val_loss: 1.1894 - val_acc: 0.5763
Epoch 3/25
50000/50000 [==============================] - 781s - loss: 1.1631 - acc: 0.5863 - val_loss: 1.0722 - val_acc: 0.6195
Epoch 4/25
50000/50000 [==============================] - 825s - loss: 1.0357 - acc: 0.6319 - val_loss: 1.0372 - val_acc: 0.6286
Epoch 5/25
50000/50000 [==============================] - 833s - loss: 0.9350 - acc: 0.6683 - val_loss: 0.9808 - val_acc: 0.6528
Epoch 6/25
50000/50000 [==============================] - 833s - loss: 0.8468 - acc: 0.7013 - val_loss: 0.9381 - val_acc: 0.6706
Epoch 7/25
50000/50000 [==============================] - 878s - loss: 0.7775 - acc: 0.7242 - val_loss: 0.9208 - val_acc: 0.6750
Epoch 8/25
50000/50000 [=======================

In [ ]:
#http://machinelearningmastery.com/object-recognition-convolutional-neural-networks-keras-deep-learning-library/